In [ ]:
import pandas as pd
import os
import time
import re
import json
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
import google.generativeai as genai
import networkx as nx

# Load environment variables
load_dotenv()

In [ ]:
class APIKeyManager:
    """
    Quản lý nhiều Google API keys và tự động chuyển đổi khi gặp lỗi.
    
    Quy tắc:
    - Mỗi key được thử tối đa 2 lần
    - Sau 2 lần lỗi → tự động chuyển key tiếp theo
    - Hết key → báo lỗi
    """
    
    MAX_RETRIES_PER_KEY = 2  # Số lần thử tối đa cho mỗi key
    
    def __init__(self):
        """Khởi tạo và load tất cả API keys từ .env"""
        # Load các API keys từ environment
        self.keys = [
            ("GOOGLE_API_KEY", os.getenv("GOOGLE_API_KEY")),
            ("GOOGLE_API_KEY_2", os.getenv("GOOGLE_API_KEY_2")),
            ("GOOGLE_API_KEY_3", os.getenv("GOOGLE_API_KEY_3")),
            ("GOOGLE_API_KEY_4", os.getenv("GOOGLE_API_KEY_4")),
        ]
        
        # Chỉ giữ lại các key hợp lệ (không None)
        self.keys = [(name, key) for name, key in self.keys if key]
        
        if not self.keys:
            raise ValueError("❌ Không tìm thấy API key! Kiểm tra file .env")
        
        # Khởi tạo trạng thái
        self.current_index = 0
        self.error_counts = {name: 0 for name, _ in self.keys}  # Đếm lỗi mỗi key
        
        print(f"✓ Phát hiện {len(self.keys)} API keys")
        self._activate_key(0)
    
    def _activate_key(self, index):
        """Kích hoạt API key tại vị trí index"""
        if index >= len(self.keys):
            raise Exception("❌ Đã hết tất cả API keys!")
        
        self.current_index = index
        key_name, key_value = self.keys[index]
        
        # Cấu hình Google AI với key mới
        genai.configure(api_key=key_value)
        
        print(f"🔑 Đang sử dụng: {key_name} (Key {index + 1}/{len(self.keys)})")
    
    def get_current_key(self):
        """Lấy API key hiện tại"""
        return self.keys[self.current_index][1]
    
    def get_models(self):
        """
        Tạo các model AI với API key hiện tại.
        
        Returns:
            tuple: (model, model_more_temp, model_pro)
        """
        current_key = self.get_current_key()
        
        model = ChatGoogleGenerativeAI(
            model="gemini-2.0-flash-lite", 
            temperature=0.02,
            google_api_key=current_key
        )
        
        model_more_temp = ChatGoogleGenerativeAI(
            model="gemini-2.0-flash-lite", 
            temperature=0.1,
            google_api_key=current_key
        )
        
        model_pro = ChatGoogleGenerativeAI(
            model="gemini-2.5-pro-exp-03-25", 
            temperature=0.1,
            google_api_key=current_key
        )
        
        return model, model_more_temp, model_pro
    
    def on_error(self):
        """
        Xử lý khi gặp lỗi API.
        
        Returns:
            bool: True nếu có thể tiếp tục (đã chuyển key hoặc còn retry),
                  False nếu đã hết key
        """
        key_name, _ = self.keys[self.current_index]
        self.error_counts[key_name] += 1
        
        error_count = self.error_counts[key_name]
        print(f"⚠ Lỗi lần {error_count} với {key_name}")
        
        # Nếu đã đạt giới hạn retry cho key này
        if error_count >= self.MAX_RETRIES_PER_KEY:
            print(f"⛔ {key_name} đã lỗi {error_count}/{self.MAX_RETRIES_PER_KEY} lần")
            
            # Thử chuyển sang key tiếp theo
            next_index = self.current_index + 1
            
            if next_index < len(self.keys):
                print(f"🔄 Chuyển sang key tiếp theo...")
                self._activate_key(next_index)
                return True  # Đã chuyển key thành công
            else:
                print("❌ ĐÃ HẾT TẤT CẢ API KEYS!")
                return False  # Không còn key nào
        
        # Vẫn còn retry cho key hiện tại
        return True
    
    def reset_error_count(self):
        """Reset đếm lỗi của key hiện tại (sau khi thành công)"""
        key_name, _ = self.keys[self.current_index]
        self.error_counts[key_name] = 0

# Khởi tạo API Manager
api_manager = APIKeyManager()

print("✓ Đã khởi tạo APIKeyManager với 4 API keys")

In [ ]:
# ============================================================
# PROMPT TEMPLATES - Trích xuất thực thể và mối quan hệ
# ============================================================

# Template trích xuất thực thể từ tin tức
entity_extraction_template = PromptTemplate.from_template("""Bạn đang làm việc dưới bối cảnh phân tích kinh tế. 
Bạn được cho một hoặc nhiều bài báo, bao gồm tựa đề và mô tả ngắn gọn về bài báo đó, ngoài ra bạn có
thông tin về ngày xuất bản của bài báo, và loại chủ đề mà bài báo đang đề cập tới.

Hạn chế tạo mới một thực thể, chỉ tạo liên kết tới 5 thực thể. Luôn ưu tiên liên kết với các thực thể đã có: {existing_entities}

Bạn cần phân tích bài báo, đưa ra tên của những thực thể (ví dụ như cổ phiếu, ngành nghề, công ty, quốc gia, tỉnh thành...)
sẽ bị ảnh hưởng trực tiếp bởi thông tin của bài báo, theo hướng tích cực hoặc tiêu cực.

Với mỗi thực thể, ở phần Tên thực thể, hạn chế dùng dấu chấm, gạch ngang, dấu và &, dấu chấm phẩy ;. Và cần ghi thêm quốc gia, địa phương cụ thể và ngành nghề của nó (nếu có).
Tên chỉ nói tới một thực thể duy nhất. Phần Tên không được quá phức tạp, đơn giản nhất có thể.
Nếu thực thể nào thuộc danh mục cổ phiếu sau: {portfolio}, hãy ghi rõ tên cổ phiếu.
Ví dụ: SSI-Chứng khoán; Ngành công nghiệp Việt Nam; Người dùng Mỹ; Ngành thép Châu Á; Ngành du lịch Hạ Long, ...

Ghi nhớ, Hạn chế tạo mới một thực thể, chỉ tạo liên kết tới 5 thực thể. Luôn cố liên kết với các thực thể đã có.

Phần giải thích mỗi thực thể, bắt buộc đánh giá số liệu được ghi, nhiều hoặc ít, tăng hoặc giảm, gấp bao nhiêu lần, ...
Cần cố gắng liên kết với nhiều thực thể khác. Tuy nhiên không suy ngoài phạm vi bài báo. Không tự chèn số liệu ngoài bài báo.
Không dùng dấu hai chấm trong phần giải thích, chỉ dùng hai chấm : để tách giữa Tên thực thể và phần giải thích.
                                                          
Đưa ra theo định dạng sau:
[[POSITIVE]]
[Entity 1]: [Explanation]
...
[Entity N]: [Explanation]

[[NEGATIVE]]
[Entity A]: [Explanation]
..
[Entity Z]: [Explanation]
                                                          
Một ví dụ cho bài báo:

(BẮT ĐẦU VÍ DỤ)

Ngày đăng: 2025-04-07T22:51:00+07:00
Loại chủ đề: Kinh tế
Tựa đề: Nỗ lực hiện thực hóa mục tiêu thông tuyến cao tốc từ Cao Bằng đến Cà Mau 

Mô tả: Nhằm hoàn thành mục tiêu đến năm 2025 cả nước có trên 3.000 km đường cao tốc, Bộ Xây dựng, các địa phương và doanh nghiệp đang triển khai thi công 28 dự án/dự án thành phần với tổng chiều dài khoảng 1.188 km. 
Đến nay, tiến độ đa số các dự án bám sát kế hoạch, nhiều dự án đăng ký hoàn thành thông tuyến trong năm 2025. Có thể nói ngành giao thông vận tải đang cố gắng hết sức.

Danh sách thực thể sẽ bị ảnh hưởng:

[[POSITIVE]]
Bộ Xây dựng Việt Nam: Áp lực quản lý 28 dự án với tổng chiều dài 1188 km, nhằm hiện thực hóa mục tiêu đạt 3000 km cao tốc vào năm 2025. Số lượng dự án tăng gấp nhiều lần so với giai đoạn trước, đòi hỏi điều phối nguồn lực và kiểm soát tiến độ chặt chẽ hơn.
Chính quyền địa phương Việt Nam: Trực tiếp phối hợp triển khai các dự án tại từng tỉnh thành. Cần nâng cao năng lực quản lý và sử dụng ngân sách công hiệu quả để đảm bảo tiến độ thi công theo kế hoạch chung quốc gia.
Doanh nghiệp xây dựng Việt Nam: Được hưởng lợi trực tiếp khi nhận khối lượng hợp đồng thi công lớn. Doanh thu và năng lực thi công có thể tăng nhanh hơn so với các giai đoạn trước đây, nhờ nhu cầu đầu tư hạ tầng tăng mạnh.

[[NEGATIVE]]
Bộ Xây dựng Việt Nam: Rủi ro chậm tiến độ và đội vốn nếu điều phối không hiệu quả do số lượng dự án tăng gấp nhiều lần.
Chính quyền địa phương Việt Nam: Có thể gặp khó khăn trong giải phóng mặt bằng và quản lý vốn đầu tư nếu năng lực tổ chức yếu.

(KẾT THÚC VÍ DỤ)

Ngày đăng: {date}
Loại chủ đề: {group}
Tựa đề: {title}

Mô tả: {description}


Danh sách thực thể sẽ bị ảnh hưởng:
""")

print("✓ Đã định nghĩa entity_extraction_template")

In [ ]:
PORTFOLIO_STOCKS = ["FPT", "SSI", "VCB", "VHM", "HPG", "GAS", "MSN", "MWG", "GVR", "VIC"]
PORTFOLIO_SECTOR = ["Công nghệ", "Chứng khoán", "Ngân hàng", "Bất động sản", "Vật liệu cơ bản", 
                     "Dịch vụ Hạ tầng", "Tiêu dùng cơ bản", "Bán lẻ", "Chế biến", "Bất đọng sản"]
BASE_DELAY = 30
MAX_RETRIES = 3

def create_chains(api_manager):
    """
    Tạo chains với models từ APIKeyManager
    """
    model, model_more_temp, model_pro = api_manager.get_models()
    
    # Tạo chain trích xuất thực thể
    chain_entity = entity_extraction_template | model
    
    return chain_entity

# Khởi tạo chain
chain_entity = create_chains(api_manager)

In [ ]:
def invoke_chain_with_retry(chain, prompt, api_manager, base_delay=BASE_DELAY):
    """
    Gọi chain với cơ chế retry tự động và tích hợp APIKeyManager
    """
    total_attempts = 0
    max_total_attempts = len(api_manager.keys) * api_manager.MAX_RETRIES_PER_KEY
    
    while total_attempts < max_total_attempts:
        try:
            # Thử gọi API
            response = chain.invoke(prompt)
            
            # Thành công -> reset error count
            api_manager.reset_error_count()
            return response
            
        except Exception as e:
            total_attempts += 1
            error_msg = str(e)
            
            # Báo lỗi cho API manager
            switched = api_manager.on_error()
            
            if total_attempts >= max_total_attempts:
                print(f"❌ Đã thử tất cả {len(api_manager.keys)} API keys ({total_attempts} lần) nhưng vẫn lỗi")
                print(f"   Lỗi cuối: {error_msg}")
                return None
            
            # Chờ trước khi retry
            if switched:
                delay = base_delay
                print(f"⏳ Đợi {delay}s trước khi thử key mới...")
            else:
                retry_num = api_manager.error_counts.get(api_manager.current_index, 0)
                delay = base_delay * (1.5 ** (retry_num - 1))
                print(f"⏳ Đợi {delay:.0f}s trước khi retry ({retry_num}/{api_manager.MAX_RETRIES_PER_KEY})...")
            
            time.sleep(delay)

def parse_entity_response(response):
    """
    Phân tích response từ entity extraction prompt
    
    Returns:
        dict: {"POSITIVE": [(entity, explanation), ...], "NEGATIVE": [(entity, explanation), ...]}
    """
    if response is None:
        print("Response is None")
        return {"POSITIVE": [], "NEGATIVE": []}
        
    sections = {"POSITIVE": [], "NEGATIVE": []}
    current_section = None
    str_resp = response.content
    
    for line in str(str_resp).splitlines():
        line = line.strip()
        if not line:
            continue
        if "[[POSITIVE]]" in line.upper():
            current_section = "POSITIVE"
            continue
        if "[[NEGATIVE]]" in line.upper():
            current_section = "NEGATIVE"
            continue
        if current_section and ':' in line:
            entity = line.split(":", 1)[0].strip()
            # Skip invalid entities
            if not entity or "không có thực thể nào" in entity.lower():
                continue
            # content = all line except entity
            content = line.split(entity, 1)[-1].strip(':').strip()
            sections[current_section].append((entity, content))

    return sections

def merge_entity(entity, canonical_set):
    """
    Trả về phiên bản canonical của entity nếu đã tồn tại (case-insensitive),
    nếu không thì thêm và trả về entity mới.
    """
    normalized_entity = str(entity).strip('[').strip(']').strip(' ').lower()
    for exist in canonical_set:
        if exist.lower() == normalized_entity:
            return exist
    canonical_set.add(normalized_entity)
    return normalized_entity

def graph_entities_to_str(G, max_entities=50):
    """
    Chuyển đổi các entities trong graph thành chuỗi để đưa vào prompt
    """
    entities = [node for node in G.nodes() if not node.startswith("Article_")]
    # Giới hạn số lượng để không làm prompt quá dài
    entities = entities[:max_entities]
    return ", ".join(entities) if entities else "Chưa có thực thể nào"

print("✓ Đã định nghĩa các hàm tiện ích")

In [ ]:
# ============================================================
# HÀM CHÍNH - Trích xuất thực thể từ tin tức
# ============================================================

def extract_entities_from_news(
    csv_path="summarized_news_with_stocks.csv",
    output_path="entities_extracted.csv",
    start_date=None,
    end_date=None,
    max_articles=None
):
    """
    Trích xuất thực thể từ tin tức đã tóm tắt
    
    Parameters:
    -----------
    csv_path : str
        Đường dẫn đến file CSV chứa tin tức đã tóm tắt
    output_path : str
        Đường dẫn file CSV kết quả
    start_date : str, optional
        Ngày bắt đầu (format: YYYY-MM-DD)
    end_date : str, optional
        Ngày kết thúc (format: YYYY-MM-DD)
    max_articles : int, optional
        Số lượng bài báo tối đa để xử lý (để test)
    
    Returns:
    --------
    tuple: (entities_df, graph, canonical_entities)
        - entities_df: DataFrame chứa các thực thể đã trích xuất
        - graph: NetworkX graph chứa mối quan hệ
        - canonical_entities: Set các thực thể canonical
    """
    print(f"📖 Đang đọc dữ liệu từ {csv_path}...")
    
    # Đọc dữ liệu tin tức đã tóm tắt
    df = pd.read_csv(csv_path)
    print(f"✓ Đã đọc {len(df)} tin tức")
    
    # Chuyển đổi cột date sang datetime
    df['parsed_date'] = pd.to_datetime(df['date'])
    df['only_date'] = df['parsed_date'].dt.date
    
    # Lọc theo khoảng thời gian
    if start_date:
        start_dt = pd.to_datetime(start_date).date()
        df = df[df['only_date'] >= start_dt]
        print(f"✓ Lọc từ ngày {start_date}: còn {len(df)} tin")
    
    if end_date:
        end_dt = pd.to_datetime(end_date).date()
        df = df[df['only_date'] <= end_dt]
        print(f"✓ Lọc đến ngày {end_date}: còn {len(df)} tin")
    
    # Giới hạn số lượng nếu cần (để test)
    if max_articles:
        df = df.head(max_articles)
        print(f"✓ Giới hạn xuống {len(df)} tin để xử lý")
    
    # Sắp xếp theo thời gian
    df = df.sort_values('date')
    
    # Khởi tạo graph và canonical entities
    G = nx.DiGraph()
    canonical_entities = set()
    
    # Build portfolio string
    portfolio_str_full = ", ".join([f"{stock}-{sector}" for stock, sector in zip(PORTFOLIO_STOCKS, PORTFOLIO_SECTOR)])
    
    # Kết quả
    all_entities = []
    
    print(f"\n{'='*60}")
    print(f"🔍 BẮT ĐẦU TRÍCH XUẤT THỰC THỂ")
    print(f"{'='*60}\n")
    
    # Xử lý từng bài báo
    for idx, row in df.iterrows():
        article_idx = idx + 1
        article_node = f"Article_{article_idx}: {row['title']}"
        article_timestamp = row['parsed_date']
        
        # Thêm node bài báo vào graph
        if not G.has_node(article_node):
            G.add_node(article_node, type="article", timestamp=article_timestamp)
        
        print(f"[{article_idx}/{len(df)}] 📰 {row['title'][:60]}...")
        
        # Tạo group từ stockCodes nếu có, không thì để "Chung"
        group = row.get('stockCodes', 'Chung') if row.get('stockCodes') else 'Chung'
        
        # Phase 1: Extract initial entities
        max_entity_retries = MAX_RETRIES
        entity_retry_count = 0
        entities_dict = {"POSITIVE": [], "NEGATIVE": []}
        
        while entity_retry_count < max_entity_retries:
            prompt_text = {
                "portfolio": portfolio_str_full,
                "date": row['date'],
                "group": group,
                "title": row['title'],
                "description": row['description'],
                "existing_entities": graph_entities_to_str(G)
            }
            
            response_text = invoke_chain_with_retry(chain_entity, prompt_text, api_manager)
            time.sleep(1)  # Rate limiting
            
            if response_text is None:
                print(f"   ❌ Bỏ qua tin {article_idx} do lỗi API")
                break
            
            entities_dict = parse_entity_response(response_text)
            
            # Check if we got any entities
            total_entities = len(entities_dict.get("POSITIVE", [])) + len(entities_dict.get("NEGATIVE", []))
            if total_entities > 0:
                print(f"   ✓ Trích xuất được {total_entities} thực thể")
                break
                
            entity_retry_count += 1
            print(f"   ⚠ Không có thực thể. Thử lại {entity_retry_count}/{max_entity_retries}")
            time.sleep(BASE_DELAY)
        
        if entity_retry_count == max_entity_retries and total_entities == 0:
            print(f"   ❌ Thất bại sau {max_entity_retries} lần thử")
            continue
        
        # Process entities
        for impact in ["POSITIVE", "NEGATIVE"]:
            for ent, content in entities_dict.get(impact, []):
                # Skip invalid entities
                if not ent or "không có thực thể nào" in ent.lower():
                    continue
                
                # Normalize entity
                canon_ent = merge_entity(ent, canonical_entities)
                
                # Determine node type
                node_type = "stock" if any(str(canon_ent).lower().find(stock.lower()) != -1 for stock in PORTFOLIO_STOCKS) else "entity"
                
                # Add node to graph
                if not G.has_node(canon_ent):
                    G.add_node(canon_ent, type=node_type, timestamp=article_timestamp)
                
                # Add edge from article to entity
                if not G.has_edge(article_node, canon_ent):
                    G.add_edge(article_node, canon_ent, impact=impact, timestamp=article_timestamp)
                
                # Lưu vào kết quả
                all_entities.append({
                    "article_id": article_idx,
                    "article_title": row['title'],
                    "date": row['date'],
                    "entity": canon_ent,
                    "entity_type": node_type,
                    "impact": impact,
                    "explanation": content
                })
    
    # Tạo DataFrame kết quả
    entities_df = pd.DataFrame(all_entities)
    
    # Lưu file
    entities_df.to_csv(output_path, index=False, encoding='utf-8-sig')
    
    print(f"\n{'='*60}")
    print(f"✅ HOÀN THÀNH!")
    print(f"📊 Tổng số entities: {len(entities_df)}")
    print(f"🔹 Unique entities: {len(canonical_entities)}")
    print(f"📈 Graph nodes: {len(G.nodes())}")
    print(f"🔗 Graph edges: {len(G.edges())}")
    print(f"💾 Đã lưu vào: {output_path}")
    print(f"{'='*60}")
    
    return entities_df, G, canonical_entities

print("✓ Đã định nghĩa hàm extract_entities_from_news()")

## 🚀 Chạy trích xuất thực thể

### Hướng dẫn sử dụng:

1. **Test với số lượng nhỏ:** Thử với `max_articles=10` để kiểm tra
2. **Chạy đầy đủ:** Bỏ `max_articles` để xử lý toàn bộ
3. **Lọc theo thời gian:** Dùng `start_date` và `end_date`

### Ví dụ:

In [ ]:
# Test với 10 tin tức đầu tiên
entities_df, G, canonical_entities = extract_entities_from_news(
    csv_path="summarized_news_with_stocks.csv",
    output_path="entities_extracted.csv",
    start_date="2022-09-30",  # Ngày bắt đầu
    end_date=None,             # None = đến cuối     
)

## 📊 Phân tích kết quả

Xem các thực thể đã trích xuất:

In [ ]:
# Xem tổng quan
print(f"📊 TỔNG QUAN KẾT QUẢ")
print(f"{'='*60}\n")

print(f"Tổng số entities trích xuất: {len(entities_df)}")
print(f"Unique entities: {len(canonical_entities)}")
print(f"Graph nodes: {len(G.nodes())}")
print(f"Graph edges: {len(G.edges())}")

print(f"\n📈 PHÂN LOẠI THEO IMPACT:\n")
print(entities_df['impact'].value_counts())

print(f"\n📌 PHÂN LOẠI THEO ENTITY TYPE:\n")
print(entities_df['entity_type'].value_counts())

print(f"\n{'='*60}")
print("MẪU 10 ENTITIES ĐẦU TIÊN:")
print(f"{'='*60}\n")

for idx, row in entities_df.head(10).iterrows():
    print(f"[{row['article_id']}] 📰 {row['article_title'][:50]}...")
    print(f"🏷️  Entity: {row['entity']} ({row['entity_type']})")
    print(f"{'✅' if row['impact'] == 'POSITIVE' else '❌'} Impact: {row['impact']}")
    print(f"📝 {row['explanation'][:100]}...")
    print(f"{'-'*60}\n")

In [ ]:
# Xem các entity được đề cập nhiều nhất
from collections import Counter

entity_counts = Counter(entities_df['entity'])

print(f"🔥 TOP 15 ENTITIES ĐƯỢC ĐỀ CẬP NHIỀU NHẤT:\n")
print(f"{'='*60}\n")

for i, (entity, count) in enumerate(entity_counts.most_common(15), 1):
    # Đếm positive và negative
    pos_count = len(entities_df[(entities_df['entity'] == entity) & (entities_df['impact'] == 'POSITIVE')])
    neg_count = len(entities_df[(entities_df['entity'] == entity) & (entities_df['impact'] == 'NEGATIVE')])
    
    print(f"{i:2d}. {entity}")
    print(f"    Tổng: {count} lần | ✅ {pos_count} | ❌ {neg_count}")
    print()

## 🔍 Tìm kiếm và phân tích

### Tìm thông tin về một entity cụ thể:

In [ ]:
def search_entity(entity_name, entities_df):
    """
    Tìm kiếm tất cả thông tin về một entity
    """
    # Tìm kiếm case-insensitive
    results = entities_df[entities_df['entity'].str.lower().str.contains(entity_name.lower(), na=False)]
    
    if len(results) == 0:
        print(f"❌ Không tìm thấy entity: {entity_name}")
        return None
    
    print(f"🔍 TÌM THẤY {len(results)} MENTIONS VỀ '{entity_name.upper()}'\n")
    print(f"{'='*60}\n")
    
    for idx, row in results.iterrows():
        print(f"[{row['article_id']}] 📰 {row['article_title']}")
        print(f"📅 {row['date']}")
        print(f"🏷️  {row['entity']} ({row['entity_type']})")
        print(f"{'✅' if row['impact'] == 'POSITIVE' else '❌'} {row['impact']}")
        print(f"📝 {row['explanation']}")
        print(f"{'-'*60}\n")
    
    return results

# Ví dụ: Tìm thông tin về FPT
# search_entity("FPT", entities_df)

In [ ]:
# Phân tích Graph: Xem các entities kết nối với nhau
def analyze_graph(G):
    """
    Phân tích knowledge graph đã xây dựng
    """
    print(f"📊 PHÂN TÍCH KNOWLEDGE GRAPH\n")
    print(f"{'='*60}\n")
    
    # Thống kê cơ bản
    print(f"Tổng số nodes: {len(G.nodes())}")
    print(f"Tổng số edges: {len(G.edges())}")
    
    # Phân loại nodes
    node_types = {}
    for node, data in G.nodes(data=True):
        node_type = data.get('type', 'unknown')
        node_types[node_type] = node_types.get(node_type, 0) + 1
    
    print(f"\n📌 Phân loại nodes:")
    for ntype, count in node_types.items():
        print(f"   {ntype}: {count}")
    
    # Tìm nodes có nhiều kết nối nhất (degree centrality)
    entity_nodes = [n for n, d in G.nodes(data=True) if d.get('type') != 'article']
    
    if entity_nodes:
        # In-degree: số lượng bài báo liên kết đến entity này
        in_degrees = [(node, G.in_degree(node)) for node in entity_nodes]
        in_degrees.sort(key=lambda x: x[1], reverse=True)
        
        print(f"\n🔥 TOP 10 ENTITIES ĐƯỢC NHẮC ĐẾN NHIỀU NHẤT (bởi các bài báo):\n")
        for i, (node, degree) in enumerate(in_degrees[:10], 1):
            print(f"{i:2d}. {node}: {degree} bài báo")
    
    print(f"\n{'='*60}")

# Chạy phân tích
analyze_graph(G)

## 💾 Lưu Graph để sử dụng sau

Lưu graph vào file pickle để sử dụng cho các bước tiếp theo (relation extraction, attention mechanism...):

In [ ]:
import pickle

# Lưu graph
with open('knowledge_graph.pkl', 'wb') as f:
    pickle.dump(G, f)
print("✓ Đã lưu knowledge graph vào 'knowledge_graph.pkl'")

# Lưu canonical entities
with open('canonical_entities.pkl', 'wb') as f:
    pickle.dump(canonical_entities, f)
print("✓ Đã lưu canonical_entities vào 'canonical_entities.pkl'")

# Để load lại sau:
# with open('knowledge_graph.pkl', 'rb') as f:
#     G = pickle.load(f)
# with open('canonical_entities.pkl', 'rb') as f:
#     canonical_entities = pickle.load(f)